# 데이터분석프로그래밍 7주차 실습

## 실습 시작 전 유의사항

※ 상단 메뉴에서 '**파일 - 드라이브에 사본 저장**' 진행 후 시작하세요.\
※ 프로젝트 제목 뒤에 본인의 이름과 학번을 작성하세요. `ex) DataAnalysisProgramming_#01_202312345_정재호.ipynb`

## 실습 개요
- 전처리
  - 정규화
- 데이터프레임 응용
  - 필터링
  - 함수 매핑
  - 데이터프레임 연결, 병합



---



## 데이터 사전처리

### 정규화
변수가 갖는 범위의 크기는 변수마다 다를 수 있습니다.

이런 경우에는 데이터 분석할 때, 큰 범위를 갖는 변수가 더 큰 영향을 갖게 됩니다.

그래서 보통 모든 변수들이 같은 크기의 범위를 갖도록 **정규화**하는 작업을 가집니다.


In [ ]:
# 6주차 때도 사용했던 planets 데이터셋
import seaborn as sns
df = sns.load_dataset('planets')

df.dropna(subset=['mass', 'distance'], axis=0, inplace=True)

df.describe() # 데이터의 정보를 요약해서 보여주는 함수

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


위의 결과에서 세 가지의 연속형 변수를 살펴봅시다.

oribital_period는 범위가 `1.32` ~ `17337.5`인 것에 비해,\
mass는 `0` ~ `25`, distance는 `46.59` ~ `354`로 범위의 크기가 다르다는 것을 알 수 있습니다.

해당 실습에서는 distance 열에 대해서만 정규화를 진행해보도록 하겠습니다.

정규화 방법으로는 데이터의 범위를 0~1으로 변환하는 Min-Max Normalization을 사용해봅시다.

In [ ]:
#Step 1. 최솟값으로 빼서 가장 작은 값이 0이 되도록 변환
df['distance'] -= df.distance.min()
# Sol: df['distance'] -= df['distance'].min()

#Step 2. 데이터의 범위 크기를 측정
min_max = df.distance.max() - df.distance.min()
# Sol: min_max = df['distance'].max() - df['distance'].min()

#Step 3. distance 열을 방금 구한 크기로 나눠 줌
df['distance'] = df['distance'] / min_max

print(df['distance'].describe())
# min과 max가 0과 1이어야 정답입니다.

count    498.000000
mean       0.143820
std        0.132131
min        0.000000
25%        0.065639
50%        0.109429
75%        0.164419
max        1.000000
Name: distance, dtype: float64


## 데이터프레임 응용

### 필터링
4주차 실습 때 나왔던 numpy의 Boolean Indexing과 유사한 개념입니다.

데이터프레임에 조건식을 이용하여 원하는 데이터만 필터링 할 수 있습니다.

In [ ]:
df.head(5)

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,0.215653,2006
1,Radial Velocity,1,874.774,2.21,0.157663,2008
2,Radial Velocity,1,763.000,2.60,0.052432,2011
3,Radial Velocity,1,326.030,19.40,0.309854,2007
4,Radial Velocity,1,516.220,10.50,0.334950,2009


여러 개의 조건을 모두 만족하는 데이터를 필터링 하려면\
조건을 & 연산자로 묶어야 합니다.

해당 실습에서 distance는 0.3보다 크고, mass는 1보다 작은 데이터를 필터링 해보세요.

In [ ]:
mask_distance = df['distance'] > 0.3
mask_mass = df['mass'] < 1

df[mask_distance & mask_mass]

,method,number,orbital_period,mass,distance,year
394,Radial Velocity,1,361.100,0.900,0.372749,2011
415,Radial Velocity,1,6.495,0.960,0.339487,2010
536,Radial Velocity,1,297.300,0.610,0.357862,2007
784,Radial Velocity,3,580.000,0.947,0.378988,2012


여러 조건을 하나라도 만족하는 데이터를 필터링 하려면\
조건을 | 연산자로 묶어야 합니다.

해당 실습에서 year이 1995, 1996, 1997 중 하나인 데이터를 필터링 해보세요.

In [ ]:
mask_1995 = df['year']==1995
mask_1996 = df['year']==1996
mask_1997 = df['year']==1997

df[mask_1995 | mask_1996 | mask_1997]

,method,number,orbital_period,mass,distance,year
13,Radial Velocity,3,1078.000000,2.5300,0.036098,1996
16,Radial Velocity,1,4.230785,0.4720,0.039728,1995
17,Radial Velocity,5,14.651000,0.8000,0.031703,1996
61,Radial Velocity,1,39.845000,1.0400,0.045598,1997
62,Radial Velocity,1,3.313500,3.9000,0.040408,1996
64,Radial Velocity,4,4.617033,0.6876,0.034368,1996


위와 같이 특정 변수가 여러가지 값 중 하나인 경우인지를 알고 싶다면\
isin 메소드를 사용할 수도 있습니다.

isin 메소드를 이용하여 방금과 같은 조건의 데이터를 필터링 해보세요.

In [ ]:
filter_year = df['year'].isin([1995, 1996, 1997])

df[filter_year]

,method,number,orbital_period,mass,distance,year
13,Radial Velocity,3,1078.000000,2.5300,0.036098,1996
16,Radial Velocity,1,4.230785,0.4720,0.039728,1995
17,Radial Velocity,5,14.651000,0.8000,0.031703,1996
61,Radial Velocity,1,39.845000,1.0400,0.045598,1997
62,Radial Velocity,1,3.313500,3.9000,0.040408,1996
64,Radial Velocity,4,4.617033,0.6876,0.034368,1996


### 함수 매핑
데이터에 행 단위, 열 단위, 요소 단위로 연산을 적용하고 싶을 수도 있습니다.

이런 경우에는 apply, applymap 메소드를 이용할 수 있습니다.

 **※ 따로 함수를 만들지 말고 lambda 함수를 이용하여 해결해봅시다. (람다 함수의 정의를 자세히 읽어 볼 것)**

In [ ]:
import pandas as pd

dict_data = {'c0':[1,2,3,4,5], 'c1':[6,7,8,9,10], 'c2':[11,12,13,14,15]}

print("원본 데이터프레임")
print(pd.DataFrame(dict_data))

원본 데이터프레임
   c0  c1  c2
0   1   6  11
1   2   7  12
2   3   8  13
3   4   9  14
4   5  10  15


In [ ]:
f = lambda x: x+3
# x는 매개변수
f(5)

8

In [ ]:
df = pd.DataFrame(dict_data)
print("원본 데이터프레임")
print(df)

# 1. axis=1인 apply 함수를 이용하여 c0열과 c2열을 합친 새로운 열 만들기
df['c0+c2'] = df.apply(lambda x : x['c0']+x['c2'], axis=1)
print("\n1. c0+c2")
print(df)

# 2. applymap 함수를 이용하여 모든 짝수 값에 100 더하기
df_apmap = df.applymap(lambda x: x+100 if x%2 == 0 else x)
print("\n2. df_apmap")
print(df_apmap)

원본 데이터프레임
   c0  c1  c2
0   1   6  11
1   2   7  12
2   3   8  13
3   4   9  14
4   5  10  15

1. c0+c2
   c0  c1  c2  c0+c2
0   1   6  11     12
1   2   7  12     14
2   3   8  13     16
3   4   9  14     18
4   5  10  15     20

2. df_apmap
    c0   c1   c2  c0+c2
0    1  106   11    112
1  102    7  112    114
2    3  108   13    116
3  104    9  114    118
4    5  110   15    120


### 데이터프레임 연결

따로 떨어져 있는 데이터프레임들을 하나로 연결해야 하는 상황이 올 수도 있습니다.

이런 경우는 concat 메소드를 이용할 수 있습니다.

In [ ]:
# 데이터셋 준비
import pandas as pd
df1 = pd.DataFrame({'a': ['o', 'o', 'o', 'o'],
                    'b': ['o', 'o', 'o', 'o'],
                    'c': ['o', 'o', 'o', 'o']},
                    index=[0, 1, 2, 3])
df2 = pd.DataFrame({'a': ['-', '-'],
                    'b': ['-', '-'],
                    'c': ['-', '-']},
                    index=[0, 1])
df3 = pd.DataFrame({'d': ['l', 'l', 'l', 'l'],
                    'e': ['l', 'l', 'l', 'l']},
                    index=[0, 1, 2, 3])

# 데이터셋 출력
print('기본 데이터', df1, '', sep='\n')
print('추가 행', df2, '', sep='\n')
print('추가 열', df3, '', sep='\n')

기본 데이터
   a  b  c
0  o  o  o
1  o  o  o
2  o  o  o
3  o  o  o

추가 행
   a  b  c
0  -  -  -
1  -  -  -

추가 열
   d  e
0  l  l
1  l  l
2  l  l
3  l  l



위 데이터셋들을 concat 메소드로 아래 예시와 같은 결과가 나오도록 연결해보세요.

- 정답 예시)
```
     a  b  c
0  o  o  o
1  o  o  o
2  o  o  o
3  o  o  o
0  -  -  -
1  -  -  -

     a  b  c
0  o  o  o
1  o  o  o
2  o  o  o
3  o  o  o
4  -  -  -
5  -  -  -

     a  b  c  d  e
0  o  o  o  l  l
1  o  o  o  l  l
2  o  o  o  l  l
3  o  o  o  l  l
```




In [ ]:
# Prob 1. 기본 데이터 df1에 추가 행 df2을 연결해보세요.
result1 = pd.concat([df1, df2], axis=0)
print(result1, '\n')

# Prob 2. result1의 index가 0,1,2,3,0,1 처럼 기존 index를 따라가지 않고, 별개의 순차적인 index를 갖도록 하세요.
result2 = pd.concat([df1, df2], axis=0, ignore_index=True)
print(result2, '\n')

# Prob 3. 기본 데이터 df1에 추가 열 df2을 연결해보세요.
result3 = pd.concat([df1, df3], axis=1)
print(result3)

   a  b  c
0  o  o  o
1  o  o  o
2  o  o  o
3  o  o  o
0  -  -  -
1  -  -  - 

   a  b  c
0  o  o  o
1  o  o  o
2  o  o  o
3  o  o  o
4  -  -  -
5  -  -  - 

   a  b  c  d  e
0  o  o  o  l  l
1  o  o  o  l  l
2  o  o  o  l  l
3  o  o  o  l  l


### 데이터프레임 병합

데이터프레임 병합은 연결과는 조금 다른 개념입니다.

두 데이터프레임을 하나로 합친다는 동작은 같지만, 특정 기준에 의해서 합쳐지는 것이 차이점입니다.

그 기준은 양쪽 데이터프레임에 동시에 존재하는 열이 됩니다.

merge 메소드를 이용해 가격 정보를 갖는 price와 재고 정보를 갖는 stock을 병합해봅시다.

In [ ]:
# 데이터셋 준비
import pandas as pd
price = pd.DataFrame({'name': ['콜라', '생수', '우유', '녹차'],
                    'price': [1200, 1000, 1300, 900]})

stock = pd.DataFrame({'name': ['우유', '생수', '김밥'],
                    'stock': [2, 10, 7]})

# 데이터셋 출력
print('가격', price, '', sep='\n')
print('재고', stock, '', sep='\n')

가격
  name  price
0   콜라   1200
1   생수   1000
2   우유   1300
3   녹차    900

재고
  name  stock
0   우유      2
1   생수     10
2   김밥      7



위 데이터셋들을 merge 함수로 아래 예시와 같은 결과가 나오도록 연결해보세요.

- 정답 예시)
```
      name  price  stock
0   생수   1000     10
1   우유   1300      2

      name   price  stock
0   콜라  1200.0    NaN
1   생수  1000.0   10.0
2   우유  1300.0    2.0
3   녹차   900.0    NaN
4   김밥     NaN    7.0

      name   price  stock
0   우유  1300.0      2
1   생수  1000.0     10
2   김밥     NaN      7
```

In [ ]:
# Prob 1. 두 데이터 price와 stock을 기본 옵션으로 병합하고 결과를 확인하세요.
res1 = pd.merge(price, stock, on='name')
# Sol:res1 = pd.merge(price, stock)
print(res1, '\n')

# Prob 2. 위 방법은 name이 양쪽에 공통으로 없는 데이터는 제거해버립니다. how 매개변수를 이용하여 제거하지 않도록 해보세요.
res2 = pd.merge(price, stock, how='outer')
print(res2, '\n')

# Prob 3. 재고(stock) 데이터가 있는 물품을 기준으로 병합하고 싶은 경우는 어떻게 해야하는가?
res3 = pd.merge(price, stock, how='right')
print(res3, '\n')

  name  price  stock
0   생수   1000     10
1   우유   1300      2 

  name   price  stock
0   콜라  1200.0    NaN
1   생수  1000.0   10.0
2   우유  1300.0    2.0
3   녹차   900.0    NaN
4   김밥     NaN    7.0 

  name   price  stock
0   우유  1300.0      2
1   생수  1000.0     10
2   김밥     NaN      7 



## 실습 종료 전 유의사항

※ 완료 시 우측 상단 프로필 옆 '**공유 - 액세스를 *링크가 있는 모든 사용자*로 전환**' 후 복사한 링크를 이루리 과제 란에 업로드 하세요.

<img src="https://drive.google.com/uc?id=1Zj1FKIz0sanqUfkfr3scp5M-dDgMFxYQ" height=300>

</br>

※ 추가로 상단 메뉴 '**파일 - 다운로드 - .ipynb**' 로 다운로드 후 첨부 파일로 업로드 하세요.

※ 즉, 과제의 텍스트란에는 링크를, 파일 첨부란에는 .ipynb 파일을 제출하십시오.